In [8]:
import numpy as np
import pandas as pd
from transformers import *
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from datasets import Dataset, load_from_disk, load_metric
import pickle
from utils.helper import read_py150k_code, read_file_to_string
import regex as re
from model.scratch_model import InRepPlusGAN

#import model file
#import 



In [12]:
#plbart init
#tokenizer = PLBartTokenizer.from_pretrained("uclanlp/plbart-python-en_XX", src_lang="python", tgt_lang="python" )
#model = PLBartForConditionalGeneration.from_pretrained("uclanlp/plbart-python-en_XX")
train_plbart_dataset = load_from_disk('datasets/plbart_train.hf')
test_plbart_dataset = load_from_disk('datasets/plbart_test.hf')
train_plbart_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_plbart_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

#codet5 init
#tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base')
#model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base')
train_codet5_dataset = load_from_disk('datasets/plbart_train.hf')
test_codet5_dataset = load_from_disk('datasets/plbart_test.hf')

train_codet5_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_codet5_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [13]:
#sample and save data subset

# sample_plbart = train_plbart_dataset.shuffle(seed=34).select(range(1000))
# sample_codet5 = train_codet5_dataset.shuffle(seed=34).select(range(1000))
# sample_plbart[0]
# sample_plbart.save_to_disk("datasets/plbart_sample.hf")
# sample_codet5.save_to_disk("datasets/codet5_sample.hf")

In [4]:
class CustomTrainer(Trainer):
    #redefine this loss this is an example
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 3.0]))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [5]:
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")
metric = load_metric("accuracy")
def compute_metrics(eval_pred): # this part prob wont work, parameter should be removed from trainer probably
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [14]:
model = InRepPlusGAN()

https://huggingface.co/uclanlp/plbart-multi_task-python/resolve/main/config.json not found in cache or force_download set to True, downloading to /soe/swade1/.cache/huggingface/transformers/tmpv2ig_30q
Downloading: 100%|██████████████████████████████| 811/811 [00:00<00:00, 429kB/s]
storing https://huggingface.co/uclanlp/plbart-multi_task-python/resolve/main/config.json in cache at /soe/swade1/.cache/huggingface/transformers/f7b387cf7785602ccd0994f177f6d66cf7a940399d7751d145c4c0e979cd6ec8.a2d21d48f57f004fa4d7bde6ae303fbc9ec8fcddc26626f1bc72809e1b5515e4
creating metadata file for /soe/swade1/.cache/huggingface/transformers/f7b387cf7785602ccd0994f177f6d66cf7a940399d7751d145c4c0e979cd6ec8.a2d21d48f57f004fa4d7bde6ae303fbc9ec8fcddc26626f1bc72809e1b5515e4
loading configuration file https://huggingface.co/uclanlp/plbart-multi_task-python/resolve/main/config.json from cache at /soe/swade1/.cache/huggingface/transformers/f7b387cf7785602ccd0994f177f6d66cf7a940399d7751d145c4c0e979cd6ec8.a2d21d48f5

OSError: Unable to load weights from pytorch checkpoint file for 'uclanlp/plbart-multi_task-python' at '/soe/swade1/.cache/huggingface/transformers/a6c1a417d9ecdadba0b3dd2152c200d79226c62bc8c2d81f4cc929f13843ae0d.0d0232267d270f46f512a2b4ca7e40fe59714ef85df037b1c6114b03c491d4b1'. If you tried to load a PyTorch model from a TF 2.0 checkpoint, please set from_tf=True.

In [15]:

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

NameError: name 'model' is not defined